# ДЗ для работы с файлами

у нас есть текст, которых лежит в файле city_smells.txt. Давайте проведем с ним элементарные количественные исследования: можно, например, узнать, сколько в тексте уникальных слов, размер самого длинного предложения и тд. Чтобы работать с текстом, который лежит в файле, нам надо:

Открыть файл (не забудьте о режиме открытия и энкодинге)
Прочитать его
Сохранить текст в переменную, с которой можно работать дальше
Предобработка текста: удалить пунктуацию, свести текст к нижнему регистру

Что можно сделать с текстом: (пункты на выбор, минимум один)

1. определить среднюю длину слова в тексте
2. определить среднюю длину предложения в тексте
3. посчитать, во сколько раз самое длинное предложение длиннее самого короткого (такое же можно сделать со словами)
4. (не убирая пунктуацию) - среднее количество пунктуационных знаков в предложении
5. количество уникальных слов и пророрция общему количеству слов в тексте
6. что-то еще, что Вы сами захотите поисследовать
Запишите результат Вашего мини-исследования в новый файл, добавьте его и (отдельный) файл с кодом в Ваш репозиторий.

In [172]:
# В Википедии приведено следующее определение предложения: "Предложе́ние (в языке) — это единица языка, которая представляет 
# собой грамматически организованное соединение слов (или слово), обладающее смысловой и интонационной законченностью.[1] С
# точки зрения пунктуации, предложение как законченная единица речи оформляется в конце точкой, восклицательным или 
# вопросительным знаками — или многоточием." В связи с этим я не учитывала слово "источники" и все, что идет после него, как
# предложения и удалила эту часть.
# Что касается пунктуационных знаков, то я исходила из того, что "<...> в русском языке встречаются такие знаки пунктуации:
# точка, многоточие, запятая, точка с запятой, скобки и кавычки, двоеточие, тире, восклицательный и вопросительный знаки", т. 
# е., например, косую черту и дефис в качестве знаков пунктуации я не учитывала.
import os, string, re
from nltk import sent_tokenize
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
morph = MorphAnalyzer()

In [173]:


# 4. Определить среднее количество пунктуационных знаков в предложении
def determine_average_sentence_punctuation_marks_number(sentence_list):
    punctuation_marks_number_list = []
    sum = 0
    for sentence in sentence_list:
        punctuation_marks = re.findall(r'[.,;()«»:—!?]', sentence)
        punctuation_marks_number = len(punctuation_marks)
        punctuation_marks_number_list.append(punctuation_marks_number)
    for number in punctuation_marks_number_list:
        sum += number
    average_punctuation_marks_number = round(sum / len(punctuation_marks_number_list), 1)
    return 'Среднее количество пунктуационных знаков в предложении: ' + str(average_punctuation_marks_number)


# 2. Определить среднюю длину предложения в тексте
def determine_average_sentence_length(sentence_list):
    sentence_length_list = []
    sum = 0
    for sentence in sentence_list:
        sentence_length_list.append(len(sentence))
        sum += len(sentence)
    average_sentence_length = round(sum / len(sentence_length_list), 1)
    return 'Средняя длина предложения (среднее количество символов с учетом пробелов): ' + str(average_sentence_length)


# 3. Посчитать, во сколько раз самое длинное предложение длиннее самого короткого
def determine_sentence_length_difference(sentence_list):
    sentence_length_list = []
    max_length = 0
    for sentence in sentence_list:
        sentence_length_list.append(len(sentence))
    min_length = sentence_length_list[0]
    for number in sentence_length_list:
        if number > max_length:
            max_length = number
        if number < min_length:
            min_length = number
    difference = round(max_length / min_length, 1)
    return 'Самое длинное предложение длиннее самого короткого в ' + str(difference) + ' раз(-а)'
    

# 5. Определить количество уникальных слов и пропорцию общему количеству слов в тексте
def determine_unique_words_number_and_proproption(text):
    lemma_list = []
    text = preprocess_text(text)
    word_list = text.lower().split()
    for elem in word_list:
        lemma_list.append(morph.parse(elem)[0].normal_form)
    # Наличие небольшого количества неверных лемм (напр., "свежевыпечь") в итоге не сказывается на качестве данной функции.
    lemma_list = list(set(lemma_list))
    unique_words_percentage = round((len(lemma_list) / len(word_list)) * 100)
    return ('Количество уникальных слов в тексте: ' + str(len(lemma_list)) + ', или ' + str(unique_words_percentage)
    + '% от общего количества слов (' + str(len(word_list)) + ' без учета метаданных)')

    
# 1. Определить среднюю длину слова в тексте
def determine_average_word_length(text):
    text = preprocess_text(text)
    word_list = text.lower().split()
    sum = 0
    for word in word_list:
        sum += len(word)
    average_word_length = round(sum / len(word_list), 1)
    return 'Средняя длина слова в тексте: ' + str(average_word_length) + ' символа (-ов)' 


# От себя: определить ключевые слова в тексте для примерного понимания его тематики
def find_key_words(text):
    text = preprocess_text(text)
    tokens = word_tokenize(text, language='russian')
    tagged = pos_tag(tokens, lang='rus')
    deletion_list = ['PR', 'CONJ', 'PART', 'INTJ', 'PRAEDIC-PRO', 'ADV-PRO', 'A-PRO=m', 'A-PRO=f', 'A-PRO=pl', 'S-PRO', \
                     'PARENTH', 'PRAEDIC', 'ADV', 'NUM=acc']
    cleared_list = []
    keywords_list = []
    stops = stopwords.words('russian')
    stops.append('есть')
    [cleared_list.append(tag) for tag in tagged if tag[1] not in deletion_list]
    words_normalized = [morph.parse(elem[0])[0].normal_form for elem in cleared_list]
    # Из-за неправильной лемматизации слова "данные" в ключевые слова попадает слово "дать". В программе оно заменяется на  
    # правильную лемму, чтобы не портить выдачу.
    for index, elem in enumerate(words_normalized):
        if elem == 'дать':
            words_normalized[index] = 'данные'        
    words_normalized = [word for word in words_normalized if word not in stops]
    for elem in words_normalized:
        if words_normalized.count(elem) >= 3:
            keywords_list.append(elem)
    keywords_list = list(set(keywords_list))
    return 'Ключевые слова: ' + ', '.join(keywords_list)


def preprocess_text(text):
    # Цифры не учитываются как слова.
    return re.sub(r'\d+', '', re.sub(r'[^\w\s-]', '', re.sub(r'/', ' ', text.lower())))
     

with open('Files\\city_smells.txt', 'r', encoding='utf-8') as input_text:
    read_input_text = input_text.read()
    
remove_index = read_input_text.index('\nИсточники:')
remade_text = read_input_text[:(remove_index - 1)]
remade_text = re.sub(r'\n+', ' ', remade_text)
sentence_list = re.split(r'(?:(?<=[.?!:])|(?<=$))\s(?:(?=[А-Я])|(?=\s))', remade_text)

In [174]:
# Сравнение работы моего разделителя по предложениям и sent_tokenize из nltk:
sent_tokenize_result = sent_tokenize(remade_text, 'russian')
print(len(sentence_list), len(sent_tokenize_result))
difference = list(set(sent_tokenize_result) - set(sentence_list))
print(difference)
# Поскольку мой вариант работает лучше для этого текста, я использую его, а не sent_tokenize.

32 30
['Цвета выбраны не просто так. Исследователи нашли связи между запахами и цветами, преобладающими на анализируемых фотографиях.', 'На странице проекта, путешествуя по интерактивным картам, можно посмотреть соотношение запахов по каждому сегменту, например, в Барселоне на улице Бальмес запах еды преобладает: Вместе запахи складываются в комплексный и сложный ландшафт, уникальный для каждого города, и он нуждается в исследовании так же, как и ландшафт визуальный или звуковой.']


In [175]:
f = open('Files\\lesson_6_results.txt', 'w', encoding = 'utf-8')
f.write(determine_average_sentence_punctuation_marks_number(sentence_list) + os.linesep)
f.write(determine_average_sentence_length(sentence_list) + os.linesep)
f.write(determine_sentence_length_difference(sentence_list) + os.linesep)
f.write(determine_unique_words_number_and_proproption(remade_text) + os.linesep)
f.write(determine_average_word_length(remade_text) + os.linesep)
f.write(find_key_words(remade_text) + os.linesep)
f.close()